# SampleFlow API examples

In [1]:
import requests
import os
import pandas as pd

In [2]:
rest_api_url = "https://circuitseq.iwr.uni-heidelberg.de/api"

## Public API

This can be accessed by anyone without requiring an authorization token.

### `/remaining`

- returns how many samples are still available this week
- doesn't require an authentication token

In [3]:
# requests is the standard Python library for this:
response = requests.get(f"{rest_api_url}/remaining")

In [4]:
# status code is 200 on success, 401 or other if something went wrong:
response.status_code

200

In [5]:
# the request returns data in json format (which is converted to a Python dict by requests):
response.json()

{'message': '', 'remaining': 96}

## Admin API

An authorization token for an admin account is required

In [6]:
# if we try to access an admin endpoint without suitable authentication we get an error:
response = requests.get(f"{rest_api_url}/admin/samples")

In [7]:
response.status_code

401

In [8]:
response.json()

{'msg': 'Missing Authorization Header'}

## Auth token
- you can get create an API token from the admin page
- then export it as an evironment variable on your computer e.g. `export CIRCUITSEQ_API_TOKEN=abca12421...`
- need to add this token to your header when making requests if you want to authenticate yourself
- NOTE: currently these tokens are invalidated whenever the code is updated & the server is restarted (which is currently quite often!)
  - if this happends your previously working authorized request will return a 422 status code with `Signature verification failed` in the json
  - generating a new token from the admin page and exporting it should fix this

In [9]:
# get the API token from the environment variable (better not to directly add it to your script)
auth_token = os.environ["CIRCUITSEQ_API_TOKEN"]
# make an authorization header for requests to use
auth_header = {"Authorization": f"Bearer {auth_token}"}

### `/admin/samples`

- returns current samples and previous samples

In [10]:
response = requests.get(f"{rest_api_url}/admin/samples", headers=auth_header)

In [11]:
response.status_code

200

In [12]:
response.json().keys()

dict_keys(['current_samples', 'previous_samples'])

In [13]:
pd.DataFrame(response.json()["current_samples"])

""


### `/admin/zipsamples`

- download a zipfile with a tsv of the samples table and reference sequence fasta files for this week

In [14]:
response = requests.post(f"{rest_api_url}/admin/zipsamples", headers=auth_header)

In [15]:
response.status_code

200

In [16]:
# write downloaded binary data to a file:
with open("samples.zip", "wb") as f:
    f.write(response.content)

### `/admin/result`

- upload success or failure status for results of sample analysis

#### Upload a successful result

- you need to provide
  - `primary_key`: the primary key of the sample
  - `file`: the zipfile of results with name `[PRIMARY_KEY]_[NAME].zip`
  - `success="true"`: success status of analysis
- zip file structure: must contain these files
  - `[PRIMARY_KEY]_[NAME].fasta`
  - `[PRIMARY_KEY]_[NAME].gbk`
- on success returns the path on the server where the zipfile is stored
  - the user will also receive an email with fasta/gbk results attached
- on failure a diagnostic message will be returned

In [21]:
with open(
    "../backend/tests/data/results/22_46_A2_ZIP_TEST_pMC_Final_Kan.zip", "rb"
) as f:
    response = requests.post(
        f"{rest_api_url}/admin/result",
        data={"primary_key": "22_46_A2", "success": "true"},
        files={"file": f},
        headers=auth_header,
    )

In [22]:
response.status_code

401

In [23]:
response.json()

{'message': 'Unknown primary key 22_46_A2'}

#### Upload an unsuccessful result

- you need to provide
  - `primary_key`: the primary key of the sample
  - `success="false"`: success status of analysis
- zip file not required
- if the primary key is valid the user will receive an email with the unsuccessful analysis status
- otherwise a diagnostic message will be returned

In [24]:
response = requests.post(
    f"{rest_api_url}/admin/result",
    data={"primary_key": "22_01_A1", "success": "false"},
    headers=auth_header,
)

In [25]:
response.status_code

401

In [26]:
response.json()

{'message': 'Unknown primary key 22_01_A1'}